### Read behavioral data file

[stimulus file format in afni](https://afni.nimh.nih.gov/afni/community/board/read.php?1,158062,158065#msg-158065)

In [1]:
import pandas as pd
import numpy as np
import re

sid = 11
sub = f"sub-{sid:04d}"
sub

'sub-0011'

In [2]:
if sid % 2 == 1:
    runs = ['01','02']
else:
    runs = ['02','01']
        
   


logfile = [f"../beh/{sub}_ses-01_task-GFORCE_run-L.csv",
          f"../beh/{sub}_ses-01_task-GFORCE_run-R.csv",
          ]


for i in range(2):
    data = pd.read_csv(logfile[i])
    #print(data.columns)
    
    target_starts = data.loc[:, ["trials.thisN","TARGET.started", "Target_Size"]].dropna().reset_index()
    isi_rec = data.loc[:, ["trials.thisN","ISItime"]].dropna().reset_index()
    
    if sid == 2:
        target_starts["onset"] = target_starts.loc[:,"TARGET.started"] - \
                data.loc[0, 'Ins_key.rt'] - data.loc[0, 'Ins_key.started'] -2.4
        target_starts.loc[:,"onset"] = np.ceil(target_starts.loc[:,"onset"])
    else:
        target_starts["onset"] = target_starts.loc[:,"TARGET.started"] - \
                data.loc[0, 'Ins_key.rt'] - data.loc[0, 'Ins_key.started']
        
            
    
    data2 = pd.merge(isi_rec, target_starts, on="trials.thisN")
    data3 = data2.loc[:,["onset","ISItime","Target_Size"]]
    data3.rename(columns={"ISItime":"duration", "Target_Size":"trial_type"}, 
                inplace=True)
    data3.loc[:,'onset'] = np.floor(data3.loc[:,'onset']).astype(int)
    data3.loc[:,'duration'] = np.floor(data3.loc[:,'duration']).astype(int)
    #data3.applymap({0.6:'40%',0.9:'60%'})
    data3['trial_type'] = data3[['trial_type']].applymap({0.6:'40%',0.9:'60%'}.get)
    data3.to_csv(re.sub("-[LR].csv",f"-{runs[i]}_events.tsv", logfile[i]), sep="\t", index=False)

FileNotFoundError: [Errno 2] No such file or directory: '../beh/sub-0011_ses-01_task-GFORCE_run-L.csv'

In [ ]:
target_starts